# Test the model

First, test the model locally and remotely

## Test locally

In [4]:
!pip install onnxruntime


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [33]:
import IPython.display as ipd

import librosa
import requests
import numpy as np
import onnxruntime as rt

In [10]:
sess = rt.InferenceSession(
    "sound_classifier.onnx", 
    providers=["CPUExecutionProvider"]
)

input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

In [12]:
print("output input: ", input_name)
print("output name: ", output_name)

output input:  mels
output name:  output


Load an audio sample and convert the signal to MELs

In [17]:
signal, sample_rate = librosa.load("samples/waves.wav")

ipd.Audio(signal, rate=sample_rate)

In [19]:
mels = np.mean(
    librosa.feature.melspectrogram(y = signal, sr = sample_rate, n_mels=128), 
    axis=1
)

input_data = np.array([mels]).astype(np.float32)

In [25]:
class_logits = sess.run([output_name], {input_name: input_data})[0]
class_logits

array([[-0.39227486, -0.6821451 ,  1.9121404 , -0.8648834 ,  0.5950277 ,
        -1.1331943 ,  1.9425615 ,  0.0715602 , -0.4858188 , -0.73726296]],
      dtype=float32)

Get the class with the highest value

In [28]:
predicted_class_idx = np.argmax(class_logits)

In [40]:
CLASSES = [
    'dog',
    'chainsaw',
    'crackling_fire',
    'helicopter',
    'rain',
    'crying_baby',
    'clock_tick',
    'sneezing',
    'rooster',
    'sea_waves'
]
class_name = CLASSES[predicted_class_idx]
print(f"Predicted class: {class_name} ({predicted_class_idx})")

Predicted class: clock_tick (6)


## Test the Deployed Model

In [38]:
INFERENCE_ENDPOINT = "https://soundclassifier-jairamir-sounds-rhods.apps.tools-na100.dev.ole.redhat.com/v2/models/soundclassifier/infer"

payload = {
    "inputs": [
        {
            # The ONNX model requires this name
            "name": "mels",
            "shape": [1, 128],
            "datatype": "FP32",
            "data": [mels.tolist()]
        }
    ]
}

response = requests.post(INFERENCE_ENDPOINT, json=payload)
result = response.json()
result

{'model_name': 'soundclassifier__isvc-82dcb689ca',
 'model_version': '1',
 'outputs': [{'name': 'output',
   'datatype': 'FP32',
   'shape': [1, 10],
   'data': [-0.39227486,
    -0.6821451,
    1.9121404,
    -0.8648834,
    0.5950277,
    -1.1331943,
    1.9425615,
    0.071560204,
    -0.4858188,
    -0.73726296]}]}

In [42]:
class_id = np.argmax(result["outputs"][0]["data"])
class_name = CLASSES[class_id]
print(f"Predicted class: {class_name} ({class_id})")

Predicted class: clock_tick (6)


For more information about how to run inference via REST, see https://github.com/kserve/modelmesh-serving/blob/main/docs/predictors/run-inference.md#inference-using-rest